<a href="https://colab.research.google.com/github/fdavenport/CIVE480A6-climate-change-impacts/blob/main/lectures/06_Analyzing_Extreme_Events.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIVE 480A6: Climate Change Risks and Impacts
## Week 9: Analyzing Extreme Events

This week's Objectives:
1. Analyze the distribution of daily maximum temperature.
2. Calculate percentiles of the temperature distribution.
3. Calculate how often daily temperatures exceed certain thresholds.
4. Calculate "block maxima" (in this case, the hottest day of the year).
5. Learn how to fit a Generalized Extreme Value (GEV) distribution to the time series of block maxima.

## Part 1: Daily Temperature Data

Today we will be looking at daily temperature data. We will again be using data from the [Global Historical climatology Network (GHCN-D)](https://www.ncei.noaa.gov/products/land-based-station/global-historical-climatology-network-daily).  

We will be looking specifically at data from a weather station near Atlanta, Georgia. The data file contains the high and low (maximum and minimum) temperatures for each day.

<img src="https://raw.githubusercontent.com/fdavenport/CIVE480A6-climate-change-impacts/main/lectures/img/atlanta_map.png" width="400">
<img src="https://raw.githubusercontent.com/fdavenport/CIVE480A6-climate-change-impacts/main/lectures/img/heatwave.jpg" width="300">

In [ ]:
# The data has already been added to the course github page at the following link:

atl_temp_data_url = "https://raw.githubusercontent.com/fdavenport/CIVE480A6-climate-change-impacts/refs/heads/main/lectures/data/USW00013874_atlanta_temp.csv"


In [ ]:
# we are working with tabular data in a .csv file, so we need to import the pandas library

import pandas as pd

In [ ]:
# read in the data

atl_data = pd.read_csv(atl_temp_data_url)

In [ ]:
# look at the data

atl_data

We see that the table contains daily data beginning in 1940 and ended in 2022. There are two variable columns that correspond to the high (max) and low (min) temperature on each day.

In [ ]:
# convert DATE column to datetime format so that we can add year and month information

atl_data["DATE"] = pd.to_datetime(atl_data["DATE"])
atl_data["year"] = atl_data["DATE"].dt.year
atl_data["month"] = atl_data["DATE"].dt.month

atl_data.set_index("DATE", inplace=True)

atl_data


In [ ]:
# get a summary of the data


atl_data.describe()

In [ ]:
# import matplotlib.pyplot to make graphs


import matplotlib.pyplot as plt


In [ ]:
# make a plot of the data


fig, ax = plt.subplots()

ax.scatter(x = atl_data.index, y = atl_data["TMAX_degC"], s = 1, color = "brown")
ax.set_title("Daily Maximum Temperature in Atlanta, GA")
ax.set_ylabel("Temperature(C)");



We see that there is a large spread in daily maximum temperature in Atlanta, with daily high temperatures ranging anywhere from -10C on the coldest days to 40C on the very hottest days.

It's hard to tell whether there are trends in the data, but it does appear that there haven't been as many extremely cold days since ~2000.

In [ ]:
# create a histogram of the daily temperature data


fig, axes = plt.subplots(1, 2, figsize = (15, 5))


axes[0].hist(x = atl_data["TMAX_degC"], bins = 30, color = "brown", edgecolor = "gray");
axes[0].set_title("Distribution of daily maximum T")
axes[0].set_xlabel("Temperature (C)")
axes[0].set_ylabel("number of days");

axes[1].hist(x = atl_data["TMIN_degC"], bins = 30, color = "tomato", edgecolor = "gray");
axes[1].set_title("Distribution of daily minimum T")
axes[1].set_xlabel("Temperature (C)")
axes[1].set_ylabel("number of days");

## Part 2: Calculating percentiles and exceedences

In this section, we are going to look at the 95th percentile minimum and maximum temperature within the summer months (June, July, and August) in Atlanta, GA. This will give us a sense for what a rare or "extreme" hot temperature would look like at this particular location.

Oftentimes, we use percentiles to define extremes, because what is extreme in one place might not be extreme in another location. For example, 100 F is very extreme in Alaska, but not so extreme in Phoenix, AZ.

First let's look at daily maximum summer temperatures in Atlanta:

In [ ]:
## subset the data for summer months

summer_data = atl_data.loc[atl_data["month"].isin([6, 7, 8])]

summer_data


We will use the [quantile()](https://numpy.org/doc/2.0/reference/generated/numpy.quantile.html) function from the numpy package to calculate the 95th percentile

In [ ]:
# import numpy

import numpy as np


In [ ]:
# for the np.quantile function,
# the first argument is our data, the second argument is the quantile (aka percentile) that we want to calculate

# calculate the 95th percentile of TMAX for all summer days:

TMAX_p95 = np.quantile(summer_data["TMAX_degC"], q = 0.95)

TMAX_p95

In [ ]:
## conver to Fahrenheit

TMAX_p95*9/5+32

This tells us that about 5% of summer days in Atlanta have temperatures above 96.08 F.

How many days have maximum temperatures above the 95th percentile? To answer this, we will compare the temperature on each day to our 95th percentile threshold to see if the value is greater than or equal to the threshold. We will first do this for the first 10 values from our table to see how this works:  

In [ ]:
# print out the first 10 rows of our table

summer_data.iloc[0:10]

In [ ]:
## compare these 10 values for
summer_data.iloc[1:10]["TMAX_degC"] >= TMAX_p95

Python returns "False" if the temperature is lower than TMAX_p95, and "True" if the temperature is greater than TMAX_p95.

Conveniently, Python equates "True" with a value of 1, and "False" with a value of 0. If we take the sum of the True and False data, the total sum will be equal to the number of True cases.

In [ ]:
sum(summer_data.iloc[1:10]["TMAX_degC"] >= TMAX_p95)

This tells us that there were two True cases within the first 10 rows. If we repeat this for all of the summer data, we will know the total number of days with temperatures greater than or equal to TMAX_p95:

In [ ]:
sum(summer_data["TMAX_degC"] >= TMAX_p95)


There are 456 days with temperatures >= 35.6 C.

In [ ]:
## get the total number of rows (aka days) in our data

len(summer_data)

In [ ]:
## calculate percentage of days with T > TMAX_p95

456/7636*100

5.97% of days have temperatures above TMAX_p95. This isn't exactly 5% because it turns out there are a lot of duplicate values in the data. Quite a few days have temperatures of exactly 35.6 C.

We can use the same function to calculate different percentiles:

In [ ]:
# the 99th percentile of maximum daily temperature
np.quantile(summer_data["TMAX_degC"], q = 0.99)

In [ ]:
# convert to Fahrenheit

37.2*9/5+32

In [ ]:
# the 1st percentile of minimum daily summer temperature
# in other words, only 1% of summer days have temperatures below this value

np.quantile(summer_data["TMIN_degC"], q = 0.01)

In [ ]:
# convert to F

13.9*9/5+32

## Part 3: Calculating changes in the frequency of extreme cold and extreme hot days

Let's calculate the frequency of extreme hot days over time to see if it has changed. We will look at summertime TMAX and TMIN.

While TMAX tells us the hottest conditions of the day, daily minimum temperatures (TMIN) are an especially important metric to understand the human health consequences of heat waves. If it stays very warm at night, people are unable to cool down, and sustained hot temperatures become more likely to cause negative health impacts.

In [ ]:
TMIN_p95 = np.quantile(summer_data["TMIN_degC"], q=0.95)


In [ ]:
## how many days are there with maximum temperatures above the 95th percentile in each year?
## Hint: let's use our code from previous lectures to loop through all of the years

annual_freq = pd.DataFrame(columns = ["year", "days_above_TMAX_p95", "days_above_TMIN_p95"], index = range(83))

for i, yr in enumerate(range(1940, 2023)):
  data_yr = summer_data[str(yr):str(yr)] # create a subset with summer data for the current year

  annual_freq.loc[i, "year"] = yr

  # check how many days exceeded the 95th percentile
  annual_freq.loc[i, "days_above_TMAX_p95"] = sum(data_yr["TMAX_degC"] >= TMAX_p95)
  annual_freq.loc[i, "days_above_TMIN_p95"] = sum(data_yr["TMIN_degC"] >= TMIN_p95)


In [ ]:
## check the results

annual_freq

In [ ]:
## make a time series plot of the change in TMAX and TMIN days above the 95th percentile:





What are some other ways we could assess changes in the intensity or frequency of extreme hot events?


*   ?
*   ?



## Part 4: Using Extreme Value statistics to analyze very rare temperature events

In this section, we will use extreme value statistics to estimate the probability of very rare extreme events, including those that may be more rare than anything in the historical data.

Recall from class, that extreme value theory applies to "block maxima", or the maximum value in each block of time. For this case, we will consider each calendar year as a block of time. This means we need to calculate the maximum value within each year. For this analysis, we will look at TMAX.

In [ ]:
## calculate annual maximum TMAX value

annual_max = pd.DataFrame(columns = ["year", "TMAX_max"], index = range(83))

for i, yr in enumerate(range(1940, 2023)):
  data_yr = summer_data[str(yr):str(yr)] # create a subset with summer data for the current year

  annual_max.loc[i, "year"] = yr
  annual_max.loc[i, "TMAX_max"] = data_yr["TMAX_degC"].max()

annual_max["TMAX_max"] = annual_max["TMAX_max"].astype("float")

In [ ]:
## plot annual maximum time series to see

fig, ax = plt.subplots()

ax.plot(annual_max["year"], annual_max["TMAX_max"], color = "gray")
ax.scatter(annual_max["year"], annual_max["TMAX_max"], color = "purple", zorder = 5)
ax.set_title("Time series of annual maxima")
ax.set_ylabel("Maximum Temperature in each year(C)");

In [ ]:
## plot histogram of annual maxima



The Fisher–Tippett–Gnedenko theorem tells us that the distribution of block maxima can be described by the Generalized Extreme Value (GEV) distribution.

The GEV distribution is described by three parameters: the location ($\mu$), the scale ($\sigma$), and the shape ($\xi$).

<img src="https://raw.githubusercontent.com/fdavenport/CIVE480A6-climate-change-impacts/main/lectures/img/GEV.png" width="500">

Just like we can calculate mean and standard deviation for a sample dataset, we can figure out the location, scale, and shape parameters that best match our data.

We will use the [genextreme()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.genextreme.html#scipy.stats.genextreme) function from the scipy package to calculate the GEV parameters for our data.  

In [ ]:
## import genextreme() function



In [ ]:
## fit our data



In [ ]:
## make a graph of a GEV distribution with these parameters



In [ ]:
## plot the histogram of our data and the GEV distribution together



Now that we have calculated the parameters of the GEV distribution, we can calculate the magnitude of events with specific return periods (or in other words, an event with a specific probability, such as 1%).

In [ ]:
## what is the magnitude of a 10-year event?



In [ ]:
## what about a 20-year event? a 100-year event? a 500-year event?



Let's make a figure showing how the event magnitude changes for different return periods:

In [ ]:
## define return periods

return_periods = np.array([1.01, 1.02, 1.05, 1.1, 1.5, 2, 5, 10, 20, 30, 40, 50, 100, 200, 500, 1000])

In [ ]:
## calculate magnitude for each return period


In [ ]:
## make a figure showing return periods



#import matplotlib.ticker as mticker
#ax.xaxis.set_major_formatter(mticker.ScalarFormatter())
#ax.ticklabel_format(style='plain', axis='x')


Let's add our data points to the plot to see how closely they match the curve. To do this, we need to calculate the *empirical* return level of each data point. The empirical return level refers to how frequently different values in our data were exceeded within the period of record.

For example, if we have 84 years of data, the most extreme event in our data has an empirical return period of 84 years because it only occurred once in 84 years. The second most extreme event has an empirical return level of 42 years because it was exceeded twice in 84 years.


In [ ]:
## The function below will calculate empirical return periods for a timeseries of annual maxima

from scipy import stats

def calc_empirical_return_level(data):
    """
    Compute empirical return level
    """
    df = pd.DataFrame(index=np.arange(data.size))
    # sort the data
    df["sorted_value"] = np.sort(data)[::-1]
    # rank via scipy instead to deal with duplicate values
    df["ranked_value"] = np.sort(stats.rankdata(-data))
    # find exceedence probability
    n = data.size
    df["exceedance"] = df["ranked_value"] / (n + 1)
    # find return period
    df["return_period"] = 1 / df["exceedance"]

    df = df[::-1]

    return df

In [ ]:
## calculate empirical return levels for our data



In class, we have talked about how extreme events are changing. This means that a 100-year event now might become a 50-year event in the future!
How might we address this if we are trying to analyze extreme event risk within a particular engineering application?  


*   ?
*   ?
*  ?



### Practice:

Practice applying your extreme value analysis to calculate the 100-year flood for the [Merced River](https://waterdata.usgs.gov/nwis/inventory/?site_no=11266500) in Yosemite National Park in California. The USGS provides annual peak data for each of it's streamgages, and the data for this location has been added to the github site at the following url:

In [ ]:
merced_river_annual_peaks_url = "https://raw.githubusercontent.com/fdavenport/CIVE480A6-climate-change-impacts/refs/heads/main/lectures/data/USGS_11266500_peak_flows.csv"

In [ ]:
## read in the data



In [ ]:
## plot a time series of the annual_maxima to check the data



In [ ]:
## calculate the GEV parameters




In [ ]:
## make a plot of the GEV distribution




In [ ]:
## make a plot of event magnitude vs. return period


